In [1]:
# conda env -n pyRAS , Python 3.9.7
import h5py, sys
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import folium
from utils import ras_output_extract_wse_to_shp
from utils import ras_output_wse_shp_to_nc
import os
import shapefile

In [2]:
# RAS HDF plan with 2d results
hf = h5py.File(r"S:\For_Myles\WestPark\WestPark.p01.hdf",'r')

home_dir = os.getcwd()
tempDir = os.path.join(home_dir,"tempfiles")
postp_area = os.path.join(home_dir, "out/postp_area.shp")

In [3]:
# Get projection wkt from hdf file
wkt = hf.attrs['Projection'].decode('utf-8')
wkt


'PROJCS["NAD_1983_StatePlane_Louisiana_North_FIPS_1701_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",3280833.333333333],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-92.5],PARAMETER["Standard_Parallel_1",31.16666666666667],PARAMETER["Standard_Parallel_2",32.66666666666666],PARAMETER["Latitude_Of_Origin",30.5],UNIT["Foot_US",0.3048006096012192],AUTHORITY["EPSG",3451]]'

In [4]:
# Get 2DArea Names, Timestep
ras_output_extract_wse_to_shp.tempDirSweep(tempDir)
list_of_2DAreas = ras_output_extract_wse_to_shp.get2DAreaNames(hf)
timesteps = ras_output_extract_wse_to_shp.get_timesteps(hf)
all_data = np.empty((0, timesteps + 6), ) # Includes an extra wse column for maximum row value


In [5]:
#Initialize shapefile of all 2D flow cells
poly_wse_shp = os.path.join(tempDir, 'ras_wse.shp')
w = shapefile.Writer(poly_wse_shp)

In [6]:
# ***********Begin writing to polygon shapefile using rows of cell index pts
# Start writing to Shapefile
# Writing field names and types
# "C": Characters, text.
# "N": Numbers, with or without decimals.
# "F": Floats(same as "N").
# "L": Logical, for boolean True / False values.
# "D": Dates.
# "M": Memo
w.field('Area2D', 'C')
w.field('Cell_Index', 'N')
w.field('Easting', 'N', decimal=2)
w.field('Northing', 'N', decimal=2)
w.field('min_elev', 'N', decimal=2)

#Creating Results fields, same number as timesteps
i = 0
while i < timesteps:
    w.field('wse_' + str(i), 'N', decimal=2)
    i += 1
# Shapefile is closed after 2DArea loop

#Add a wse for maximum water surface at end
w.field('wse_max', 'N', decimal=2)

In [7]:
#Loop through all 2D flow areas in HDF file and extract geometry pts and results
for curr_2DArea in list_of_2DAreas:
    print("Current 2D Area is: %s" % curr_2DArea)

    xy_pts = np.array(ras_output_extract_wse_to_shp.get2DArea_cellcenter_pts(curr_2DArea, hf))
    min_elev = np.array(ras_output_extract_wse_to_shp.get2DCells_min_elev(curr_2DArea, hf)).round(decimals=2)
    # transpose_min_elev = min_elev.T

    wse_data = np.array(ras_output_extract_wse_to_shp.get2DArea_wse_data(curr_2DArea, hf))
    transpose_wse = wse_data.T.round(decimals=2)

    #Find WSE values that are equal to cell min elev, set to NaN, all others set to 1
    repeats_cell_min_elev = np.tile(min_elev, (timesteps,1)).T
    cell_depths = transpose_wse - repeats_cell_min_elev
    cell_depths[cell_depths > 0] = 1
    cell_depths[cell_depths == 0] = 0

    #Remove zero depth values
    filtered_transpose_wse = cell_depths * transpose_wse
    filtered_transpose_wse[filtered_transpose_wse==0] = -9999
    filtered_transpose_wse.round(decimals=2)
    max_of_row = np.max(filtered_transpose_wse, axis=1)

    cell_index = np.arange(xy_pts.shape[0])
    curr_2DArea_index = [curr_2DArea.decode('UTF-8')]* (xy_pts.shape[0])

    #Adding columns to results array
    all_data_for_curr_2DArea = np.column_stack((curr_2DArea_index,cell_index, xy_pts, min_elev))
    all_data_for_curr_2DArea = np.concatenate((all_data_for_curr_2DArea, filtered_transpose_wse), axis=1)
    all_data_for_curr_2DArea = np.column_stack((all_data_for_curr_2DArea, max_of_row))

    #Save into the overall dataset
    all_data = np.append(all_data, all_data_for_curr_2DArea, axis=0)

    # Assemble 2D Cell Polygons
    cell_face_info = ras_output_extract_wse_to_shp.get_Cells_Face_Info(hf, curr_2DArea)
    cell_face_xy_pts = ras_output_extract_wse_to_shp.get_FacePoints_Coordinates(hf, curr_2DArea)
    cell_face_index_pts = ras_output_extract_wse_to_shp.get_Cells_FacePoints_Index(hf, curr_2DArea)

    #Assemble info about perimeter faces and facepoints
    cell_facept_is_perimeter = ras_output_extract_wse_to_shp.is_FacePoint_perimeter(hf, curr_2DArea)
    face_facept_index = ras_output_extract_wse_to_shp.get_faces_FacePoint_Index(hf, curr_2DArea)
    face_perimeter_info = ras_output_extract_wse_to_shp.get_faces_Perimeter_Info(hf, curr_2DArea)
    face_perimeter_values = ras_output_extract_wse_to_shp.get_faces_Perimeter_Values(hf, curr_2DArea)
    face_orientation_info = ras_output_extract_wse_to_shp.get_face_orientation_info(hf, curr_2DArea)
    face_orientation_values = ras_output_extract_wse_to_shp.get_face_orientation_values(hf, curr_2DArea)

    #Assemble current polygons
    cell_ids = []
    index_size = len(cell_face_index_pts[0])
    curr_2DArea_Polygon_xy_pts = []
    cell_id = 0
    cell_ids = []
    for row in cell_face_index_pts:
        #find if facepoints are perimeter
        perimeter_facepts = []
        
        for facept in row:
            
            if facept != -1:

                if cell_facept_is_perimeter[facept] == -1:
                    perimeter_facepts.append(facept)
        #print(perimeter_facepts)

        #Declare empty polygon list for 2D cell
        polygon = []
        i = 0
        while i < index_size:
            curr_facept = row[i]
            
            if curr_facept != -1:
                polygon.append(cell_face_xy_pts[curr_facept])

            if i < (index_size -1) :
                next_facept = row[i+1]

            if i == (index_size -1):
                next_facept = row[0]


            if curr_facept in perimeter_facepts:
                
                if next_facept in perimeter_facepts:
                    face_index=0
                    
                    for face in face_facept_index:
                        if curr_facept == face_facept_index[face_index][0]:
                            potential_face = face_index
                            
                            if next_facept == face_facept_index[potential_face][1]:
                                next_is_first = False
                                curr_face_index = face_index
                                # print("found face")
                                break

                        if next_facept == face_facept_index[face_index][0]:
                            potential_face = face_index
                            
                            if curr_facept == face_facept_index[potential_face][1]:
                                next_is_first = True
                                curr_face_index = face_index
                                # print("found face")
                                break

                        face_index +=1


                    perimeter_st_pt = face_perimeter_info[curr_face_index][0]
                    num_perimeter_pts = face_perimeter_info[curr_face_index][1]
                    perimeter_end_pt = perimeter_st_pt + num_perimeter_pts - 1
                    perimeter_pt_index = perimeter_st_pt

                    extra_perimeter_xy_pts = []

                    # print("...adding perimeter pts, for face %s" % curr_face_index )
                    while perimeter_pt_index <= perimeter_end_pt:
                        # polygon.append(face_perimeter_values[perimeter_pt_index])
                        extra_perimeter_xy_pts.append(face_perimeter_values[perimeter_pt_index])
                        perimeter_pt_index += 1

                    if next_is_first:
                        extra_perimeter_xy_pts = extra_perimeter_xy_pts[::-1]

                    polygon.extend(extra_perimeter_xy_pts)
                #polygon.append(cell_face_xy_pts[next_facept])

            i += 1

        #Append the first face pt coordinate
        polygon.append(cell_face_xy_pts[row[0]])

        #Append to the total 2D Area set if more than 2 points (there are some lateral weirs represented like this)
        if sum(1 for n in row if n != -1)>=3:
            curr_2DArea_Polygon_xy_pts.append(polygon)
            #Keep track of cell_ids that make it into the polygon set
            cell_ids.append(cell_id)

        cell_id += 1

    #--------------Saving polygons and records to shapefile-------------
    print ("writing %s polygons to shapefile..." %curr_2DArea.decode('UTF-8'))
    str_curr_2DArea = curr_2DArea.decode('UTF-8')
    
    for row_id, poly_row in enumerate(curr_2DArea_Polygon_xy_pts):
        
        if len(poly_row) > 2:
            w.poly([poly_row[::-1]]) #clockwise flip
            #w.record(INT=nr, LOWPREC=nr, MEDPREC=nr, HIGH)
            #w.record(Area2D=str_curr_2DArea,Cell_Index=cell_id, Easting=all_data_for_curr_2DArea[cell_id][])
            #w.record('Area2D', str_curr_2DArea)
            #w.record('Cell Index', cell_id)
            records = np.array(all_data_for_curr_2DArea[cell_ids[row_id]]).tolist()
            w.record(*records)
print ('Done.')

Current 2D Area is: b'2D_Area'
writing 2D_Area polygons to shapefile...
Done.


In [8]:
# Close 2DArea polygon shapefile
print("Closing shapefile with all 2D Area polygons.")
w.close()

print("Writing Projection file w/ hardcoded coordinate system.")
with open(os.path.join(tempDir,'ras_wse.prj'), 'w') as f:
    f.write(wkt)
    f.close()

hf.close()

Closing shapefile with all 2D Area polygons.
Writing Projection file w/ hardcoded coordinate system.


In [9]:
gdf = gpd.GeoDataFrame()

# m = folium.Map([30.85, -93.3], zoom_start=14, tiles='cartodbdark_matter')
f = folium.Figure(width=800, height=550)
m = folium.Map(location=[30.87, -93.29],tiles='cartodbpositron', zoom_start=13, min_zoom = 10).add_to(f)
# m = folium.Map(location=[30.87, -93.29], tiles='cartodbdark_matter',zoom_start=14, min_zoom = 10).add_to(f)
# f


In [10]:
wse_gdf = gpd.read_file(poly_wse_shp)

In [11]:
wse_gdf.head()

,Area2D,Cell_Index,Easting,Northing,min_elev,wse_0,wse_1,wse_2,wse_3,wse_4,...,wse_41,wse_42,wse_43,wse_44,wse_45,wse_46,wse_47,wse_48,wse_max,geometry
0,2D_Area,0.0,3025166.74,144146.16,220.39,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,"POLYGON ((3025216.743 144096.158, 3025116.743 ..."
1,2D_Area,1.0,3025266.74,144146.16,220.39,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,"POLYGON ((3025316.743 144096.158, 3025216.743 ..."
2,2D_Area,2.0,3025366.74,144146.16,220.28,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,"POLYGON ((3025316.743 144187.072, 3025428.046 ..."
3,2D_Area,3.0,3023966.74,144046.16,218.83,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,"POLYGON ((3023845.898 144067.003, 3024016.743 ..."
4,2D_Area,4.0,3024066.74,144046.16,219.19,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,"POLYGON ((3024016.743 144080.950, 3024116.743 ..."


In [12]:
style_function = lambda x: {
    "color": "#E5214E"
    if x["properties"]["wse_max"] == -9999.0
    else "#1E63A0"
}

In [13]:
# g = folium.GeoJson(data=wse_gdf, style_function=style_function).add_to(m)
# folium.GeoJsonTooltip(fields=["wse_max"]).add_to(g)
# m
# g.data



In [14]:
# Clip shp to max wse bounds
wse_clip = wse_gdf[wse_gdf['wse_max'] != -9999.0]
print('.')



.


In [15]:
g = folium.GeoJson(data=wse_clip, style_function=style_function).add_to(m)
folium.GeoJsonTooltip(fields=["wse_max"]).add_to(g)
m

In [30]:
# Get max_wse boundary as shp.
# Create column with all equal values to use as dissolve field
wse_clip.assign(dissolve=1)
# dissolve
wse_max_boundary = wse_clip.dissolve(by='dissolve')
wse_max_boundary = wse_max_boundary.drop(wse_max_boundary.columns[1:], axis=1)
wse_max_boundary.to_file(os.path.join(tempDir,'wse_max_boundary.shp'))




,geometry
dissolve,
1,"POLYGON ((3030216.743 131496.158, 3030116.743 ..."


In [16]:
# Get mesh geometry

# Convert to 4326 coordinates

# Create timeseries from coordinates

In [17]:
# create structured grid, assign values

# Export to vector

# View values in webmap